In [1]:
#Lame Wadipeba
#Customer Data Churn Analysis

import pandas as pd

#loading all excel files into dataframes
services = pd.read_excel("Telco_customer_churn_services.xlsx")
status = pd.read_excel("Telco_customer_churn_status.xlsx")
demographics = pd.read_excel("Telco_customer_churn_demographics.xlsx")
location = pd.read_excel("Telco_customer_churn_location.xlsx")
population = pd.read_excel("Telco_customer_churn_population.xlsx")
churn = pd.read_excel("CustomerChurn.xlsx")

#dropping 'count' column to avoid duplicate merge conflicts
for df in [services, status, demographics, location]:
    if 'Count' in df.columns:
        df.drop(columns=['Count'], inplace=True)

#Merge dataframes based on Customer ID
merged_df = services.merge(location, on='Customer ID')\
                    .merge(demographics, on='Customer ID')\

#Merge Population dataset based on Zip Code
merged_df = merged_df.merge(population, on='Zip Code')

#Keep only the Satisfaction score from Status
status_subset = status[['Customer ID','Satisfaction Score','Churn Reason']]
merged_df = merged_df.merge(status_subset, on='Customer ID', how = 'left')

#Keep only the churn column from CustmerChurn and merge it in
churn_subset = churn[['Customer ID', 'Churn']]
final_df = merged_df.merge(churn_subset, on='Customer ID', how = 'left')

#fixing missing values
final_df['Internet Type'] = final_df['Internet Type'].fillna('None')
#fixing missing values
#final_df['Internet Type'] = final_df['Internet Type'].fillna('None')

#dropping columns
cols_to_drop = [
    'Internet Service','Number of Referrals', 'Service ID','Quarter', 'Location ID', 'ID',
    'Offer', 'Lat Long', 'Zip Code', 'Country', 'Population', 'Number of Dependents',
    'Monthly Charge', 'Avg Monthly Long Distance Charges', 'Total Revenue','State',
    'Total Long Distance Charges','Total Refunds', 'Total Extra Data Charges'
]

final_df = final_df.drop(columns=[col for col in cols_to_drop if col in final_df.columns])

final_df.info()

#saving csv and downloading for future use
final_df.to_csv("final_customer_churn.csv", index=False)

from google.colab import files
files.download("final_customer_churn.csv")

#Finding how many customers churned
final_df['Churn'].value_counts()





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Customer ID              7043 non-null   object 
 1   Referred a Friend        7043 non-null   object 
 2   Tenure in Months         7043 non-null   int64  
 3   Phone Service            7043 non-null   object 
 4   Multiple Lines           7043 non-null   object 
 5   Internet Type            7043 non-null   object 
 6   Avg Monthly GB Download  7043 non-null   int64  
 7   Online Security          7043 non-null   object 
 8   Online Backup            7043 non-null   object 
 9   Device Protection Plan   7043 non-null   object 
 10  Premium Tech Support     7043 non-null   object 
 11  Streaming TV             7043 non-null   object 
 12  Streaming Movies         7043 non-null   object 
 13  Streaming Music          7043 non-null   object 
 14  Unlimited Data          

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,count
Churn,
No,5174
Yes,1869


In [26]:
#Running a Logistic Regression to determine significance of variables
import pandas as pd
import statsmodels.api as sm
import numpy as np

#load cleaned csv
customer_churn = pd.read_csv("final_customer_churn (2).csv")

#dropping non predictive values, e.g customer ID, Churn Reason
columns_to_drop = ['Customer ID', 'City', 'Churn Reason','Longitude','Latitude']
df = customer_churn.drop(columns=[col for col in columns_to_drop if col in df.columns])

#Mapping Binary Columns
binary_cols = ['Multiple Lines','Phone Service', 'Online Security','Online Backup', 'Device Protection Plan',
               'Premium Tech Support','Streaming TV','Streaming Movies','Streaming Music','Unlimited Data',
               'Paperless Billing','Senior Citizen','Married','Dependents','Churn','Under 30','Referred a Friend']

for col in binary_cols:
    df[col] = df[col].map({'Yes': 1, 'No': 0})

df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})

# One-hot encode multiclass columns
multiclass_cols = ['Internet Type', 'Contract', 'Payment Method']
df = pd.get_dummies(df, columns=multiclass_cols, drop_first=True)
df = df.astype(int)

#Set up dependent and independent variables
X = df.drop(columns='Churn')
y = df['Churn']

print(X.dtypes[X.dtypes == 'object'])

print(df)

X = sm.add_constant(X)
model = sm.Logit(y, X)
result = model.fit()

print(result.summary())



         Current function value: 0.097361
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                  Churn   No. Observations:                 7043
Model:                          Logit   Df Residuals:                     7014
Method:                           MLE   Df Model:                           28
Date:                Tue, 06 May 2025   Pseudo R-squ.:                  0.8317
Time:                        00:07:39   Log-Likelihood:                -685.71
converged:                      False   LL-Null:                       -4075.1
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          65.7964   1979.956      0.033      0.973   -3814.846    3946.439
Referred a Fri

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
